In [11]:
import os
from PIL import Image, ImageOps, ImageDraw

def create_marker_shape(size):
    """
    Create a raindrop (map marker) shaped mask for the image.

    Args:
        size (tuple): The size of the marker image (width, height).

    Returns:
        PIL.Image: A mask image with a raindrop shape.
    """
    width, height = size
    mask = Image.new('L', size, 0)  # Create a blank grayscale image

    draw = ImageDraw.Draw(mask)

    # Draw the rounded top part (circle)
    circle_diameter = width
    circle_radius = circle_diameter // 2
    draw.ellipse([(0, 0), (width, circle_diameter)], fill=255)

    # Draw the pointed bottom part
    draw.polygon([
        (circle_radius, height),           # Bottom center (sharp point)
        (0, circle_radius),                # Top left of the circle
        (width, circle_radius)             # Top right of the circle
    ], fill=255)

    return mask

def create_markers(input_folder, output_folder, marker_size=(150, 250)):
    """
    Create raindrop-shaped marker images from images in the input folder
    and save them to the output folder, maintaining proper orientation.

    Args:
        input_folder (str): Path to the folder containing the original images.
        output_folder (str): Path to the folder to save the marker images.
        marker_size (tuple): Size of the markers (width, height).
    """
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Generate the raindrop-shaped mask
    mask = create_marker_shape(marker_size)

    # Loop through all files in the input folder
    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)

        # Check if the file is an image
        if os.path.isfile(input_file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')):
            try:
                # Open the image
                with Image.open(input_file_path) as img:
                    # Correct orientation based on EXIF metadata
                    img = ImageOps.exif_transpose(img)

                    # Convert RGBA to RGB if needed (e.g., for PNG with transparency)
                    if img.mode == 'RGBA':
                        img = img.convert('RGB')
                    
                    # Resize and crop the image to fit the marker shape
                    marker_image = ImageOps.fit(img, marker_size, method=Image.Resampling.LANCZOS)

                    # Apply the raindrop mask
                    marker_image.putalpha(mask)  # Add alpha channel using the mask

                    # Generate output file path
                    output_file_path = os.path.join(output_folder, file_name)

                    # Save the marker image as PNG (to preserve transparency)
                    marker_image.save(output_file_path, 'PNG')

                    print(f"Marker created for: {file_name}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

# Example Usage
input_folder_path = "./photos/Bouddha"
output_folder_path = "./markers/Bouddha"
create_markers(input_folder_path, output_folder_path, marker_size=(75, 125))


Marker created for: Tashi SamtenlingMonastery.png
Marker created for: AjimaTemple.png
Marker created for: TsechenSedup Ling Sakya Monastery.png
Marker created for: GhyoiLisangPond.png
Marker created for: BoudhaStupa1.png
Marker created for: BoudhaStupa.png
Marker created for: JhamchenLakhangMonastery.JPG
Marker created for: ThranguMonastery.JPG
Marker created for: GuruLakhangMonastery.png
Marker created for: ThranguMonastery.png
Marker created for: ThranguMonastery (2).JPG
Marker created for: Hitti (2).png
Marker created for: Hitti.png
Marker created for: AjimaTemple1.png
Marker created for: JhamchenLakhangMonastery1.JPG
